# Imports

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchinfo import summary
from tqdm import tqdm
import wandb
from torchvision.models import resnet50, ResNet50_Weights
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Functions

## Loading the pre-trained Model ResNet50 - 
## Making it compatible with iNaturalist Data - Freezing & Unfreezing layers
Code for Question 1 (Part B)

Question 1

In most DL applications, instead of training a model from scratch, you would use a model pre-trained on a similar/related task/dataset. From torchvision, you can load ANY ONE model (GoogLeNet, InceptionV3, ResNet50, VGG, EfficientNetV2, VisionTransformer etc.) pre-trained on the ImageNet dataset. Given that ImageNet also contains many animal images, it stands to reason that using a model pre-trained on ImageNet maybe helpful for this task.

You will load a pre-trained model and then fine-tune it using the naturalist data that you used in the previous question. Simply put, instead of randomly initialising the weigths of a network you will use the weights resulting from training the model on the ImageNet data (torchvision directly provides these weights). Please answer the following questions:

The dimensions of the images in your data may not be the same as that in the ImageNet data. How will you address this?
ImageNet has 1000 classes and hence the last layer of the pre-trained model would have 1000 nodes. However, the naturalist dataset has only 101010 classes. How will you address this?
(Note: This question is only to check the implementation. The subsequent questions will talk about how exactly you will do the fine-tuning.)

In [3]:
def load_and_freeze(k=3):
# Load the model and weights 
    model=resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
    model=resnet50(weights="IMAGENET1K_V1")
    model=resnet50(pretrained=True)  # deprecated
    model=resnet50(True)  # deprecated
    
    num_filters = model.fc.in_features
    model.fc = nn.Linear(num_filters, 10)
    
# Freeze all layers
    for param in model.parameters():
        param.requires_grad = False
    
# Unfreeze last k layers
    for layer in list(model.children())[-k:]:
        for param in layer.parameters():
            param.requires_grad = True
    return model

## Loading the data

In [4]:
def get_dataloaders(dir='/kaggle/input/nature-12k/inaturalist_12K/train',split=0.2,batch_size=32):
    labels = datasets.ImageFolder(root=dir).targets
    
    val_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])
    train_transforms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2,
                            saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])  
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=split, random_state=42)
    train_idx, val_idx = next(splitter.split(torch.zeros(len(labels)), labels))

    train_dataset=datasets.ImageFolder(root=dir,transform=train_transforms)
    val_dataset=datasets.ImageFolder(root=dir,transform=val_transforms)

    train_dataset = Subset(train_dataset, train_idx)
    val_dataset = Subset(val_dataset, val_idx)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader,val_loader

def test_dataloader(dir='/kaggle/input/nature-12k/inaturalist_12K/val',batch_size=32):
    labels = datasets.ImageFolder(root=dir).targets
    
    all_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])
    
    test_dataset=datasets.ImageFolder(root=dir,transform=all_transforms)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=2)

    return test_loader

## Optimizer

In [5]:
def get_optimizer(optim, lr, model):
    if optim == 'momentum':
        return torch.optim.SGD(model.parameters(), lr=lr, weight_decay=1e-4, momentum=0.9)
    elif optim == 'adamw':
        return torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)

# Finetuning
Code for Question 3 (Part B)

Question 3

Now fine-tune the model using ANY ONE of the listed strategies that you discussed above. Based on these experiments write down some insightful inferences comparing training from scratch and fine-tuning a large pre-trained model.

## Sweeps to get the right optimizer, learning rate and Batch size

In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 5
    },
    'parameters': {
        'batch_size':{
            'values':[32,64]
        },
        'optimizer':{
            'values':['momentum','adamw']
        },
        'learning_rate':{
            'min':0.0001,
            'max':0.0005
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project="DA6401_Assign2", entity="ishita49-indian-institute-of-technology-madras")

## k=1 (Unfreezing all layers except last 1 layer)

In [ ]:
def train():
    config_defaults = {
        'batch_size':64,
        'optimizer':'AdamW',
        'learning_rate':0.0004
    }
    wandb.init(config=config_defaults, tags="Updated")
    config = wandb.config
    
    model=load_and_freeze(k=1)
    
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer=get_optimizer(optim=config.optimizer,lr=config.learning_rate,model=model)
    train_loader,val_loader=get_dataloaders(batch_size=config.batch_size)
    # Train
    num_epochs=20
    print("training...")
    for epoch in range(num_epochs):
        model.train()
        total_loss=0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss+=loss.item()
        avg_loss=total_loss/len(train_loader)
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, avg_loss))
        wandb.log({"epoch": epoch + 1, "train_loss": avg_loss})
    
        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
        accuracy = 100 * correct / total
        print('Validation Accuracy: {:.2f}%'.format(accuracy))
    
        wandb.log({"val_accuracy": accuracy})
    wandb.finish()

In [ ]:
wandb.agent(sweep_id, function=train,count=10)

## k=2 (Unfreezing all layers except last 2 layers)

Taking best cnfiguration from sweep: 

Optimizer: AdamW

Learning Rate: 5e-5

Batch Size: 32

In [ ]:
def train(): 
    wandb.init(project="DA6401_Assign2")

    model=load_and_freeze(k=2)
    
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer=get_optimizer(optim='adamw',lr=5e-4,model=model)
    train_loader,val_loader=get_dataloaders(batch_size=64)
    # Train
    num_epochs=20
    print("training...")
    for epoch in range(num_epochs):
        model.train()
        total_loss=0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss+=loss.item()
        avg_loss=total_loss/len(train_loader)
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, avg_loss))
        wandb.log({"epoch": epoch + 1, "train_loss": avg_loss})
    
        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
        accuracy = 100 * correct / total
        print('Validation Accuracy: {:.2f}%'.format(accuracy))
        wandb.log({"val_accuracy": accuracy})
    wandb.finish()

    test_loader=test_dataloader(batch_size=32)

    # Test
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print('Test Accuracy: {:.2f}%'.format(accuracy))

train()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ishita49 (ishita49-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 207MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Ple

Using 2 GPUs
training...


Epoch [1/20], Loss: 1.6541


Validation Accuracy: 68.85%


...

Epoch [19/20], Loss: 0.9999


Validation Accuracy: 75.50%


Epoch [20/20], Loss: 1.0012
Validation Accuracy: 75.60%


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,█▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▄▆▆▇▅▇▆▆▇▆▇▆███▇██
epoch,20
train_loss,1.00122
val_accuracy,75.6


Test Accuracy: 74.70%


## k=3 (Unfreezing all layers except last 3 layers)

In [ ]:
def train(): 
    wandb.init(project="DA6401_Assign2")

    model=load_and_freeze(k=3)
    
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer=get_optimizer(optim='adamw',lr=5e-4,model=model)
    train_loader,val_loader=get_dataloaders(batch_size=64)
    # Train
    num_epochs=20
    print("training...")
    for epoch in range(num_epochs):
        model.train()
        total_loss=0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss+=loss.item()
        avg_loss=total_loss/len(train_loader)
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, avg_loss))
        wandb.log({"epoch": epoch + 1, "train_loss": avg_loss})
    
        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
        accuracy = 100 * correct / total
        print('Validation Accuracy: {:.2f}%'.format(accuracy))
        wandb.log({"val_accuracy": accuracy})
    wandb.finish()

    test_loader=test_dataloader(batch_size=32)

    # Test
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print('Test Accuracy: {:.2f}%'.format(accuracy))

train()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Using 2 GPUs
training...


Epoch [1/20], Loss: 1.3014


Validation Accuracy: 70.95%


...

Validation Accuracy: 78.40%


Epoch [20/20], Loss: 0.5391


Validation Accuracy: 77.00%


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,█▆▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁
val_accuracy,▁▃▅▅▄▄▆▆▄▇▅▆█▆█▅▇▆█▆
epoch,20
train_loss,0.53912
val_accuracy,77


Test Accuracy: 76.90%


## k=4 (Unfreezing all layers except last 4 layers)

In [ ]:
def train(): 
    wandb.init(project="DA6401_Assign2")

    model=load_and_freeze(k=4)
    
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer=get_optimizer(optim='adamw',lr=5e-4,model=model)
    train_loader,val_loader=get_dataloaders(batch_size=64)
    # Train
    num_epochs=20
    print("training...")
    for epoch in range(num_epochs):
        model.train()
        total_loss=0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss+=loss.item()
        avg_loss=total_loss/len(train_loader)
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, avg_loss))
        wandb.log({"epoch": epoch + 1, "train_loss": avg_loss})
    
        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
        accuracy = 100 * correct / total
        print('Validation Accuracy: {:.2f}%'.format(accuracy))
        wandb.log({"val_accuracy": accuracy})
    wandb.finish()

    test_loader=test_dataloader(batch_size=32)

    # Test
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print('Test Accuracy: {:.2f}%'.format(accuracy))

train()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Using 2 GPUs
training...


Epoch [1/20], Loss: 1.5323


Validation Accuracy: 60.45%


...

Validation Accuracy: 73.95%


Epoch [20/20], Loss: 0.6812


Validation Accuracy: 74.15%


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,█▆▅▅▄▄▃▃▃▃▃▃▂▂▂▂▁▁▁▁
val_accuracy,▁▂▅▂▄▄▄▄▆▅▆▆▇▆▅█▇▆██
epoch,20
train_loss,0.68117
val_accuracy,74.15


Test Accuracy: 73.20%


## k=5 (Unfreezing all layers except last 5 layer)

In [ ]:
def train(): 
    wandb.init(project="DA6401_Assign2")

    model=load_and_freeze(k=5)
    
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer=get_optimizer(optim='adamw',lr=5e-4,model=model)
    train_loader,val_loader=get_dataloaders(batch_size=64)
    # Train
    num_epochs=20
    print("training...")
    for epoch in range(num_epochs):
        model.train()
        total_loss=0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss+=loss.item()
        avg_loss=total_loss/len(train_loader)
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, avg_loss))
        wandb.log({"epoch": epoch + 1, "train_loss": avg_loss})
    
        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
        accuracy = 100 * correct / total
        print('Validation Accuracy: {:.2f}%'.format(accuracy))
        wandb.log({"val_accuracy": accuracy})
    wandb.finish()

    test_loader=test_dataloader(batch_size=32)

    # Test
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print('Test Accuracy: {:.2f}%'.format(accuracy))

train()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Using 2 GPUs


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


training...


Epoch [1/20], Loss: 1.6516


...

Validation Accuracy: 63.75%


Epoch [20/20], Loss: 0.7886


Validation Accuracy: 69.25%


wandb: ERROR Control-C detected -- Run data was not synced


KeyboardInterrupt: 

# References

In [ ]:
References={
    'Loading Pre-trained model':'https://pytorch.org/vision/stable/models.html',
    'Freezing Layers':'https://medium.com/we-talk-data/guide-to-freezing-layers-in-pytorch-best-practices-and-practical-examples-8e644e7a9598'
           }